In [27]:
import utils as u
import torch
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
data = pd.read_csv('./dataset/anchor_points_dataset/data_points.csv')

In [ ]:
# Reconstruct data

def reconstruct_data(data):
    data_tensor = torch.tensor(data[0:204])
    data_restructured = torch.zeros(size=(3,68)) 
    data_restructured[0, :] = data_tensor[0:68]
    data_restructured[1, :] = data_tensor[68:136]
    data_restructured[2, :] = data_tensor[136:204]
    return data_restructured


columns = [f'feature_{i}' for i in range(1, 202)] + ['label']
df_fft = pd.DataFrame(columns=columns)

for index, row in data.iterrows():
    
    row_list = row.values.flatten().tolist()
    label = row_list[-1]
    points = reconstruct_data(row_list)

    points = u.min_max_scale_data(u.rotate_face(points))
    # fft
    fft_vector = u.fft_feature_vector(points)
    df_fft.loc[index] = fft_vector.reshape(-1).tolist() + [label]

    # 

df_fft.to_csv('./dataset/anchor_points_dataset/' + 'fft_data_points.csv', index=False)



